In [8]:
from Bio import SeqIO
import pandas as pd

def extractHLAfromFasta(filename):
    """ ดึงข้อมูล HLA ออกจาก fasta file แล้ว return list of HLA Alleles """
    HLAnomenclature_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        HLAnomenclature_list.append(HLAnomenclature)
    return HLAnomenclature_list

def extractGenefromHLA(listofHLA):
    """ รับ list of HLA alleles, แล้ว return list of Gene"""
    gene_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        gene_list.append(gene)
    return gene_list

def returnuniquefromlist(list):
    """ รับ list ใด ๆ แล้ว return unique values of the list"""
    unique_list = []
    loaded_list = pd.DataFrame(list).drop_duplicates()
    for lab,row in loaded_list.iterrows():
        unique_list.append(row.item())
    return unique_list

def extractHLAfromGenelist(listofHLA, listofgene):
    """ รับ list of gene แล้ว return list of HLA ที่มียีนนั้น """
    return_list = []
    for HLA in listofHLA:
        gene = HLA.split("*")[0]
        if gene in listofgene:
            return_list.append(HLA)
        else:
            continue
    return return_list

In [9]:
# ดึงข้อมูลจาก fasta
fasta_HLA = extractHLAfromFasta('../../dataset/dataset_hla_prot.fasta')
fasta_gene = extractGenefromHLA(fasta_HLA)
fasta_unique_gene = returnuniquefromlist(fasta_gene)
fasta_unique_HLA = returnuniquefromlist(fasta_HLA)

In [10]:
# ดึงข้อมูล dataset
dataset_HLA = pd.read_csv('../../dataset/dataset.csv')

In [11]:
# #ดึงข้อมูลจาก datasets
# file = open('../3 - Cleaning & Transforming Data/log/log_notclean_Allele.txt', 'r')
# dataset_HLA = []
# for x in file.readlines():
#     x = x.split()
#     allele = x[0]
#     dataset_HLA.append(allele)
# dataset_HLA.sort()
# dataset_HLA = list(dataset_HLA[1:])
# file.close()

# dataset_gene = extractGenefromHLA(dataset_HLA)
# dataset_unique_HLA = returnuniquefromlist(dataset_HLA)
# dataset_unique_gene = returnuniquefromlist(dataset_gene)

In [12]:
dataset_gene_list = []
for x in dataset_HLA['Allele'].drop_duplicates().tolist():
    dataset_gene_list.append(x.split("*")[0])
dataset_unique_gene = set(dataset_gene_list)

print(f"ในฐานข้อมูล fasta จะมียีนทั้งหมด {len(fasta_unique_gene)} แต่ใน dataset เราจะมี {len(dataset_unique_gene)}")

#หาว่า gene อะไรที่มีทั้งใน dataset และ fasta (จะได้ทำ alignment ของยีนพวกนี้)
final_gene = list(set(fasta_unique_gene) & set(dataset_unique_gene))
final_gene.sort()
final_classone_gene, final_classtwo_gene = final_gene[0:3], final_gene[3:]
print(final_classone_gene, final_classtwo_gene)




ในฐานข้อมูล fasta จะมียีนทั้งหมด 26 แต่ใน dataset เราจะมี 12
['A', 'B', 'C'] ['DPB1', 'DQA1', 'DQB1', 'DRB1', 'DRB3', 'DRB4', 'DRB5']


In [13]:
# ดึงเอาแต่ข้อมูล HLA ที่ตรงกับ gene list ที่ใส่เข้าไป
# จะได้ข้อมูล Alleles ที่จะนำมาทำ Alignments
class1HLAtoallignment = extractHLAfromGenelist(fasta_HLA, final_classone_gene)
class2HLAtoallignment = extractHLAfromGenelist(fasta_HLA, final_classtwo_gene)

In [14]:
def createnewfastafromHLAlist_separatedgene(filename, HLAtoalignment, outputname):
    """ สร้าง HLA ใหม่โดยใช้ HLA list"""
    new_list = []
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        if HLAnomenclature in HLAtoalignment:
            new_list.append(seq_record)
    return SeqIO.write(new_list, outputname, "fasta")

def createnewfastafromGenelist(fastafile, ListofGeneWantedinfile):
    """ สร้าง fasta ใหม่โดยใช้ list gene ที่ต้องการในไฟล์นั้น ๆ """
    new_list = []
    for gene in ListofGeneWantedinfile:
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(fastafile, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            if GenefromHLAnomenclature == gene:
                new_list.append(seq_record)
    SeqIO.write(new_list, f'gene_filtered_{ListofGeneWantedinfile}.fasta', "fasta")
    print(f"Saved for gene: {ListofGeneWantedinfile}")
    print("All Done")

def dropuniqueHLAvalue(fastafile, fieldwanted,dropchars):
    """ ลบ HLA ที่ซ้ำกันออกจาก fasta file โดย field wanted = 1 ถึง 4"""
    new_df = pd.DataFrame(columns=['HLA','field1','field2','field3','field4'])
    seq_record_list = []
    
    total_counts = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        total_counts += 1
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[1]
        HLAnomenclature_splited = GenefromHLAnomenclature.split(':')

        #Add - if field is empty
        while len(HLAnomenclature_splited) < 4:
            HLAnomenclature_splited.append('-')

        to_add = pd.DataFrame({'HLA': HLAnomenclature,
        'field1':HLAnomenclature_splited[0],
        'field2':HLAnomenclature_splited[1],
        'field3':HLAnomenclature_splited[2],
        'field4':HLAnomenclature_splited[3]}, index=[0])

        new_df = new_df.append(to_add, ignore_index=True)
    new_df = new_df.reset_index(drop=True)
    df_dropped = new_df.drop_duplicates(subset=[f'field{x+1}' for x in range(fieldwanted)], keep='first')

    # Process drop chars
    if dropchars == True:
        to_drop = []
        #counter = 0
        for x in df_dropped.iterrows():
            index = -1
            y = x[-1][index][-1]
            while(y == '-'):
                index -= 1
                y = x[-1][index][-1]
            if y in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']:   
                to_drop.append(x[0])
            #counter += 1
        df_dropped = df_dropped.drop(to_drop)

    # Save file
    index_counter = 0
    for seq_record in SeqIO.parse(fastafile, "fasta"):  
        if index_counter in df_dropped.index:
            seq_record_list.append(seq_record)
        index_counter += 1

    # print(new_df)
    # print(df_dropped)
    # print(df_dropped.index)
    # # print(new_list)
    # print(seq_record_list)
    # f = open(f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.txt', "w")
    # f.write()
    # f.close()
    SeqIO.write(seq_record_list, f'{fastafile}_dropdupe{fieldwanted}field_dropchar{str(dropchars)}.fasta', "fasta")
    
    #print(f"Saved for gene: {ListofGeneWantedinfile}")
    print(f"{total_counts} were removed to {len(seq_record_list)} records")
    print("All Done")

#สร้าง csv จาก gene list
def createnewfastafromGenelist_separatedgene_csv(filename, Genetoallignment):
    for gene in Genetoallignment:
        new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
        print(f"Processing for gene: {gene}")
        for seq_record in SeqIO.parse(filename, "fasta"):  
            seq_description_list = seq_record.description.split()
            HLAnomenclature = seq_description_list[1]
            GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
            allele = seq_record.description.split()[1]
            sequence = repr(seq_record.seq).split("'")[1]
            if GenefromHLAnomenclature == gene:
                new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence},ignore_index=True)
        print(f"Saved for gene: {gene}")
        new_csv.to_csv(f'gene_{gene}.csv')
    print("All Done")

# แปลง fasta เป็น csv
def convertfastatocsv(filename):
    """ Input file in fasta format to convert to csv """
    new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
    for seq_record in SeqIO.parse(filename, "fasta"):  
        seq_description_list = seq_record.description.split()
        HLAnomenclature = seq_description_list[1]
        GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
        allele = seq_record.description.split()[1]
        sequence = ""
        for x in list(seq_record.seq):
            sequence = sequence+x
        new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
    new_csv.to_csv(f'{filename}.csv',index=False)

# def convertfastatocsv_unalligned(filename):
#     """ Input file in fasta format to convert to csv """
#     new_csv = pd.DataFrame(columns=['Alleles','Sequence'])
#     for seq_record in SeqIO.parse(filename, "fasta"):  
#         seq_description_list = seq_record.description.split()
#         HLAnomenclature = seq_description_list[1]
#         GenefromHLAnomenclature = HLAnomenclature.split('*')[0]
#         allele = seq_record.description.split()[1]
#         sequence = ""
#         for x in list(seq_record.seq):
#             sequence = sequence+x
#         new_csv = new_csv.append({"Alleles":allele,"Sequence":sequence,"Char":len(sequence)},ignore_index=True)
#     new_csv.to_csv(f'gene_{GenefromHLAnomenclature}_hla_prot.csv',index=False)


In [15]:
createnewfastafromGenelist('../../dataset/dataset_hla_prot.fasta', final_classone_gene)

Processing for gene: A
Processing for gene: B
Processing for gene: C
Saved for gene: ['A', 'B', 'C']
All Done


In [26]:
dropuniqueHLAvalue("gene_filtered_['A', 'B', 'C'].fasta", 2, dropchars=True)

24543 were removed to 12370 records
All Done


# Not used anymore

In [28]:
# createnewfastafromGenelist('hla_prot.fasta', final_classone_gene)
# createnewfastafromGenelist('hla_prot.fasta', final_classtwo_gene)

In [29]:
# #batch covert all aligned fasta file to csv
# for x in final_gene:
#     convertfastatocsv(f'gene_{x}_hla_prot_aligned.fasta')

In [30]:
# #batch covert all UNALLIGNED fasta file to csv
# #เพื่อ select ว่าอันไหนน่าจะเป้นตัวแทนที่ดีที่สุดในการเลือก
# for x in final_gene:
#     convertfastatocsv_unalligned(f'gene_{x}_hla_prot.fasta')
#     df = pd.read_csv(f'gene_{x}_hla_prot.csv')
#     try:
#         max = df[df['Char'] == df['Char'].max()].values[[0,1],[0,2]]
#         min = df[df['Char'] == df['Char'].min()].values[[0,1],[0,2]]
#     except:
#         max = df[df['Char'] == df['Char'].max()].values[0][[0,2]]
#         min = df[df['Char'] == df['Char'].min()].values[0][[0,2]]
#     print(max)
#     print(min)
#     print("with average of " + str(df['Char'].mean()))
#     print("---------------------------------")

In [31]:
# def returnHLAtwofirstfield(HLAlist):
#     return_list = []
#     for HLA in HLAlist:
#         splitedHLA = HLA.split(":")
#         HLAtwo = splitedHLA[0:2]
#         newHLA = ':'.join(HLAtwo)
#         return_list.append(newHLA)
#     return return_list

In [32]:
# #HLA 2 field แรกจากฐานข้อมูล fasta
# returnuniquefromlist(returnHLAtwofirstfield(fasta_HLA))